# Docker Swarm with Blinkt!

Was ist Docker?

![Container](https://www.docker.com/sites/default/files/Container%402x.png)

Containers are an abstraction at the app layer that packages code and dependencies together. Multiple containers can run on the same machine and share the OS kernel with other containers, each running as isolated processes in user space. Containers take up less space than VMs (container images are typically tens of MBs in size), and start almost instantly. 

![VMs](https://www.docker.com/sites/default/files/VM%402x.png)

Virtual machines (VMs) are an abstraction of physical hardware turning one server into many servers. The hypervisor allows multiple VMs to run on a single machine. Each VM includes a full copy of an operating system, one or more apps, necessary binaries and libraries - taking up tens of GBs. VMs can also be slow to boot.

## Init Docker Swarm on Cluster

Zunächst die IP-Adresse feststellen.

In [1]:
ip addr show eth0

2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether b8:27:eb:27:c2:35 brd ff:ff:ff:ff:ff:ff
    inet 172.24.2.1/24 brd 172.24.2.255 scope global eth0
       valid_lft forever preferred_lft forever
    inet 192.168.56.192/24 brd 192.168.56.255 scope global eth0
       valid_lft forever preferred_lft forever
    inet6 fd00::ba27:ebff:fe27:c235/64 scope global mngtmpaddr dynamic 
       valid_lft 7148sec preferred_lft 3548sec
    inet6 fe80::ba27:ebff:fe27:c235/64 scope link 
       valid_lft forever preferred_lft forever


In [2]:
ip addr show eth0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1 | grep 172

172.24.2.1


In [3]:
IP_LEADER=$(ip a s eth0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1 | grep  172)
echo $IP_LEADER

172.24.2.1


In [4]:
IP_LEADER_WLAN=$(ip a s wlan0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1)
echo $IP_LEADER_WLAN

172.24.1.1


In [18]:
docker node ls || docker swarm init --advertise-addr ${IP_LEADER} && docker node ls

+ docker node ls
Error response from daemon: This node is not a swarm manager. Use "docker swarm init" or "docker swarm join" to connect this node to swarm and try again.
+ docker swarm init --advertise-addr 172.24.2.1
Swarm initialized: current node (ugcpx4yfubk6t7db7q0mpxom2) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-9mfuh5lur7l7xoxlg313a2n7f 172.24.2.1:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.

+ docker node ls
ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
ugcpx4yfubk6t7db7q0mpxom2 *   cluster-00          Ready               Active              Leader


: 1

In [19]:
JOIN_TOKEN=$(docker swarm join-token -q worker)
echo $JOIN_TOKEN

++ docker swarm join-token -q worker
+ JOIN_TOKEN=SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-9mfuh5lur7l7xoxlg313a2n7f
+ echo SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-9mfuh5lur7l7xoxlg313a2n7f
SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-9mfuh5lur7l7xoxlg313a2n7f


: 1

In [20]:
JOIN_TOKEN_MANAGER=$(docker swarm join-token -q manager)
echo $JOIN_TOKEN_MANAGER

++ docker swarm join-token -q manager
+ JOIN_TOKEN_MANAGER=SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-85qfq5zcnmrdnbsqy5htj5k0k
+ echo SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-85qfq5zcnmrdnbsqy5htj5k0k
SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-85qfq5zcnmrdnbsqy5htj5k0k


: 1

## SSH einrichten

In [8]:
# Setup ssh-key for user if not exists
ls ~/.ssh/id_rsa || ansible -i localhost, -m shell -a 'ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N "" creates=~/.ssh/id_rsa' --connection=local localhost

/home/pi/.ssh/id_rsa


Den SSH Zugang vorbereiten. Zunächst aus `knwon_hosts` löschen. Dann wieder bekannt geben und den ssh-key übertragen.

In [9]:
ping -c 1 cluster-01.local

PING cluster-01.local (172.24.2.2) 56(84) bytes of data.
64 bytes from 172.24.2.2: icmp_seq=1 ttl=64 time=0.852 ms

--- cluster-01.local ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.852/0.852/0.852/0.000 ms


In [10]:
ping -c 1 cluster-01.local | grep PING | cut -d '(' -f 2 | cut -d')' -f 1

172.24.2.2


In [11]:
for i in 01 02 03 04; do
    # Hostname only
    CL_HOSTNAME=cluster-${i}
    # Get Ip via local Domain (avahi)
    CL_IP=$(ping -c 1 ${CL_HOSTNAME}.local | grep PING | cut -d '(' -f 2 | cut -d')' -f 1)
    # Cluster Hostname only
    ssh-keygen -R ${CL_HOSTNAME}
    # Cluster with Domain local
    ssh-keygen -R ${CL_HOSTNAME}.local
    # IP Cluster 
    ssh-keygen -R ${CL_IP}
    # Add Cluster into known_hosts
    ssh-keyscan -H ${CL_HOSTNAME}.local >> ~/.ssh/known_hosts
    ssh-keyscan -H ${CL_HOSTNAME} >> ~/.ssh/known_hosts
    ssh-keyscan -H ${CL_IP} >> ~/.ssh/known_hosts
    # SSH key copy into Cluster
    sshpass -p raspberry ssh-copy-id pi@${CL_HOSTNAME}.local
done

/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# Host cluster-01.local found: line 35 type RSA
# Host cluster-01.local found: line 36 type ECDSA
# Host cluster-01.local found: line 37 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# Host 172.24.2.2 found: line 35 type ED25519
# Host 172.24.2.2 found: line 36 type RSA
# Host 172.24.2.2 found: line 37 type ECDSA
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# 172.24.2.2 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# 172.24.2.2 SSH-2.0-OpenSSH_6.7p1 Raspbian-

## Dem Swarm beitreten

In [21]:
docker node ls

+ docker node ls
ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
ugcpx4yfubk6t7db7q0mpxom2 *   cluster-00          Ready               Active              Leader


: 1

Nur der Client 01 und 03 als Worker dem Swarm beitreten.

In [26]:
#set -x
for i in 01 03; do
    # Hostname only
    CL_HOSTNAME=cluster-${i}.local
    echo $CL_HOSTNAME
    ssh $CL_HOSTNAME "docker swarm join --token $JOIN_TOKEN $IP_LEADER:2377"
done
#set +x

+ for i in 01 03
+ CL_HOSTNAME=cluster-01.local
+ echo cluster-01.local
cluster-01.local
+ ssh cluster-01.local 'docker swarm join --token SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-9mfuh5lur7l7xoxlg313a2n7f 172.24.2.1:2377'
This node joined a swarm as a worker.
+ for i in 01 03
+ CL_HOSTNAME=cluster-03.local
+ echo cluster-03.local
cluster-03.local
+ ssh cluster-03.local 'docker swarm join --token SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-9mfuh5lur7l7xoxlg313a2n7f 172.24.2.1:2377'
Error response from daemon: This node is already part of a swarm. Use "docker swarm leave" to leave this swarm and join another one.


: 1

In [27]:
docker node ls

+ docker node ls
ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
ugcpx4yfubk6t7db7q0mpxom2 *   cluster-00          Ready               Active              Leader
ka0dfxhvbag77ntfgd9khjfau     cluster-01          Ready               Active              
jb1aby9ts5431iwtil5fclw9c     cluster-02          Ready               Active              Reachable
iv7y5mae02lt10tabbsy4cf71     cluster-03          Ready               Active              
jacutj39mit02u59ogc92aaks     cluster-04          Ready               Active              Reachable


: 1

In [24]:
for i in 02 04; do
    # Hostname only
    CL_HOSTNAME=cluster-${i}.local
    echo $CL_HOSTNAME
    ssh $CL_HOSTNAME "docker swarm join --token $JOIN_TOKEN_MANAGER $IP_LEADER:2377"
done

+ set -x
+ for i in 02 04
+ CL_HOSTNAME=cluster-02.local
+ echo cluster-02.local
cluster-02.local
+ ssh cluster-02.local 'docker swarm join --token SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-85qfq5zcnmrdnbsqy5htj5k0k 172.24.2.1:2377'
This node joined a swarm as a manager.
+ for i in 02 04
+ CL_HOSTNAME=cluster-04.local
+ echo cluster-04.local
cluster-04.local
+ ssh cluster-04.local 'docker swarm join --token SWMTKN-1-4km9bpw27yny1njsxc60luhl3l2wtqtrz0l2meafc64y1n8zwh-85qfq5zcnmrdnbsqy5htj5k0k 172.24.2.1:2377'
This node joined a swarm as a manager.


: 1

In [25]:
docker node ls

+ docker node ls
ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
ugcpx4yfubk6t7db7q0mpxom2 *   cluster-00          Ready               Active              Leader
jb1aby9ts5431iwtil5fclw9c     cluster-02          Ready               Active              Reachable
iv7y5mae02lt10tabbsy4cf71     cluster-03          Ready               Active              
jacutj39mit02u59ogc92aaks     cluster-04          Ready               Active              Reachable


: 1

## Visualizer anschauen

Einen graphischen Output als Service starten.

In [ ]:
docker service create \
  --name=visualizer \
  --publish=8000:8080/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  --no-resolve-image \
  --detach=false \
  alexellis2/visualizer-arm

In [ ]:
echo "http://${IP_LEADER_WLAN}:8000"
echo "http://${IP_LEADER}:8000"

In [ ]:
docker service ls

Auf allen nodes den Monitor Dienst installieren. Dieser überwacht, wie viele whoami Container laufen.

Grün: Ein whoami-Container wird gestartet.

Rot: Ein whoami-Container stoppt.

Gelb: Für die Version 1.1.0

Blau: Für die Version 1.2.0

In [ ]:
docker service create --name monitor --mode global \
  --restart-condition any --mount type=bind,src=/sys,dst=/sys \
  --mount type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  --no-resolve-image \
  --detach=false \
  stefanscherer/monitor:1.1.0

Es läuft noch kein whoami.

In [ ]:
docker service ls

In [ ]:
docker service create --name whoami \
  --no-resolve-image \
  --detach=false \
  stefanscherer/whoami:1.1.0 

In [ ]:
docker service ls

In [ ]:
docker service update \
  --detach=false \
  --image stefanscherer/whoami:1.2.0 whoami

In [ ]:
docker service scale --detach=false whoami=5

In [ ]:
docker service ls

In [ ]:
docker service update \
  --detach=false \
  --image stefanscherer/whoami:1.1.0 whoami

In [ ]:
docker service scale --detach=false whoami=20

In [ ]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.2.0 whoami

In [ ]:
docker service ls

In [ ]:
docker service scale --detach=false whoami=40

In [ ]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.1.0 whoami

In [ ]:
docker service ls

In [ ]:
docker service scale --detach=false whoami=1

In [ ]:
docker service scale --detach=false whoami=15

In [ ]:
docker service ls

In [ ]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.2.0 whoami

In [ ]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.1.0 whoami

In [ ]:
docker service ls

In [ ]:
END=5
for i in $(seq 1 $END); do
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    docker service scale --detach=false whoami=1 
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
    docker service scale --detach=false whoami=15
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
    docker service scale --detach=false whoami=40
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
done

In [ ]:
jupyter nbconvert Jupyter\ Slides.ipynb --to slides --post serve

# Aufräumen

In [ ]:
docker service rm whoami

In [ ]:
docker service rm monitor

In [ ]:
docker node ls

In [ ]:
docker service ls

In [ ]:
for i in 01 02 03 04; do
    ssh cluster-${i}.local "docker swarm leave --force"
done

In [ ]:
docker service ls

In [ ]:
docker node ls

In [ ]:
docker service rm visualizer

In [ ]:
docker swarm leave --force

In [ ]:
for i in 01 02 03 04 00; do
    ssh cluster-${i}.local "sudo reboot &"
done